# Data load

In [ ]:
import pandas as pd

path_300k = "/kaggle/input/corpus-parallel-tat-ru-300k/tat_ru_300k_parallel_corpus.csv"
corpus_par = pd.read_csv(path_300k, index_col=False)

corpus_par.drop(['Unnamed: 0'], axis=1, inplace=True)
corpus_par

,tat,ru
0,шайтан алгыры!,"а, черт!"
1,тагын ут сүнде.,опять свет выключили.
2,хәзер тәмамлыйм.,- сейчас заканчиваю.
3,"планета әйләнә, түгәрәк, әйләнмә.","планета вертится, круглая, круглая"
4,"әйе, өч тапкыр.","ну да, три."
...,...,...
311284,петро кулын михаилның иңбашына салды һәм башы ...,петро притронулся рукой к плечу михаила и кивн...
311285,"ул өстенә чибәр генә кара костюм кигән, юлдан ...","он давно уже приметил пожилого, одетого в добр..."
311286,сөрелгән җиргә җиткәч ул юлга борылды.,"дойдя до вспаханной земли, человек свернул на ..."
311287,"ул сугышчыларның сак карашларын сизеп алды да,...",он встретился с настороженными взглядами бойцо...


In [ ]:
len(corpus_par)

311289

 # Model

In [ ]:
! pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

In [ ]:
corpus_par.head(3)

,tat,ru
0,шайтан алгыры!,"а, черт!"
1,тагын ут сүнде.,опять свет выключили.
2,хәзер тәмамлыйм.,- сейчас заканчиваю.


In [ ]:
corpus_par['tat']

0                                            шайтан алгыры!
1                                           тагын ут сүнде.
2                                          хәзер тәмамлыйм.
3                         планета әйләнә, түгәрәк, әйләнмә.
4                                           әйе, өч тапкыр.
                                ...                        
311284    петро кулын михаилның иңбашына салды һәм башы ...
311285    ул өстенә чибәр генә кара костюм кигән, юлдан ...
311286               сөрелгән җиргә җиткәч ул юлга борылды.
311287    ул сугышчыларның сак карашларын сизеп алды да,...
311288    ул үз бригадасының хезмәтенә һәм уңышларына бә...
Name: tat, Length: 311289, dtype: object

In [ ]:
from datasets import Dataset, concatenate_datasets

# Convert pandas DataFrame to Hugging Face Dataset

corpus_par = corpus_par.rename(columns={'ru': 'rus_Cyrl', 'tat': 'tat_Cyrl'})

hf_dataset = Dataset.from_pandas(corpus_par)
check = Dataset.from_pandas(corpus_par[:500])

In [ ]:
hf_dataset

Dataset({
    features: ['tat_Cyrl', 'rus_Cyrl'],
    num_rows: 311289
})

In [ ]:
check

Dataset({
    features: ['tat_Cyrl', 'rus_Cyrl'],
    num_rows: 500
})

In [ ]:
from transformers import NllbTokenizerFast

checkpoint = "facebook/nllb-200-distilled-600M"
# checkpoint = "/kaggle/input/ru_tat_base_epoch1/pytorch/default/1/trainer_output/checkpoint-38389"
source_lang = "rus_Cyrl"
target_lang = "tat_Cyrl"
transl_direction = "ru->tat"
tokenizer = NllbTokenizerFast.from_pretrained(checkpoint, src_lang=source_lang, tgt_lang=target_lang)


def preprocess_function(examples, source_lang, target_lang, tokenizer):
    source = examples[source_lang]
    targ = examples[target_lang]

    model_inputs = tokenizer(
        source,
        text_target=targ,
        max_length=256,
        truncation=True,
        padding=True,
        return_tensors="pt"  # Optional, but recommended for training
    )

    return model_inputs

tokenized_corpus = hf_dataset.map(lambda x : preprocess_function(x, source_lang, target_lang, tokenizer), batched=True, remove_columns= ["rus_Cyrl", "tat_Cyrl"])

Map:   0%|          | 0/311289 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_corpus)

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

2025-06-03 11:05:53.774474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748948753.979583      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748948754.038445      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Evaluate

In [ ]:
!pip install evaluate sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.3 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Train

In [ ]:
test_pairs_size = 4180
test_size_part = test_pairs_size / len(tokenized_corpus_ru_tat)

tokenized_corpus_splited = tokenized_corpus.train_test_split(test_size=test_size_part, shuffle=True, seed=111)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainerCallback
import json

class MetricLoggerCallback(TrainerCallback):
    def __init__(self, output_path="metrics.json"):
        self.output_path = output_path
        self.metrics = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero:
            self.metrics.append(logs.copy())
            print(self.metrics)
            with open(self.output_path, "w") as f:
                json.dump(self.metrics, f, indent=2)


In [ ]:
training_args_ru_tat = Seq2SeqTrainingArguments(
    eval_strategy="steps",          # Evaluate at intervals
    eval_steps=3000,                       # Evaluate every 250 steps
    logging_strategy="steps",             # Log metrics at intervals
    logging_steps=500,                    # Log loss every 100 steps
    save_strategy="steps",                # Optional: save checkpoints
    save_steps=21000,                       # Optional: save every 500 steps


    learning_rate=8e-6,                    # Уменьшить на 60%
    weight_decay=0.005,                    # Компромиссное значение
    lr_scheduler_type='cosine',            # Плавное уменьшение LR
    warmup_ratio=0.05,

    # learning_rate=2e-5,
    # per_device_train_batch_size=8,
    # weight_decay=0.01,

    # generation_num_beams=8,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

tokenizer = tokenizer_ru_tat
trainer_ru_tat = Seq2SeqTrainer(
    model=model,
    args=training_args_ru_tat,
    train_dataset=tokenized_corpus["train"],
    eval_dataset=tokenized_corpus["test"],
    tokenizer=tokenizer_ru_tat,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[MetricLoggerCallback("ru_tat_metrics.json")]
)

print("---Training started!!!---")
trainer_ru_tat.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_19/2806679296.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_ru_tat = Seq2SeqTrainer(


---Training started!!!---


Step,Training Loss,Validation Loss,Bleu,Gen Len
3000,0.456100,0.493260,26.180600,21.443100
6000,0.461500,0.491549,26.169600,21.030400
9000,0.446000,0.491324,26.131200,21.278900
12000,0.463800,0.488963,26.348000,21.352900
15000,0.464700,0.486499,26.336700,21.112000
18000,0.477000,0.484354,26.528600,21.236400
21000,0.485100,0.481274,26.470400,20.958100
24000,0.488600,0.479951,26.562500,21.034700
27000,0.483800,0.479266,26.528400,21.218400
30000,0.498000,0.478031,26.613500,21.165600


[{'loss': 0.4941, 'grad_norm': 0.8615570664405823, 'learning_rate': 2.0791666666666665e-06, 'epoch': 0.013024564328323218}]
[{'loss': 0.4941, 'grad_norm': 0.8615570664405823, 'learning_rate': 2.0791666666666665e-06, 'epoch': 0.013024564328323218}, {'loss': 0.4246, 'grad_norm': 1.0494062900543213, 'learning_rate': 4.1624999999999995e-06, 'epoch': 0.026049128656646436}]
[{'loss': 0.4941, 'grad_norm': 0.8615570664405823, 'learning_rate': 2.0791666666666665e-06, 'epoch': 0.013024564328323218}, {'loss': 0.4246, 'grad_norm': 1.0494062900543213, 'learning_rate': 4.1624999999999995e-06, 'epoch': 0.026049128656646436}, {'loss': 0.4508, 'grad_norm': 0.9204182624816895, 'learning_rate': 6.245833333333333e-06, 'epoch': 0.039073692984969655}]
[{'loss': 0.4941, 'grad_norm': 0.8615570664405823, 'learning_rate': 2.0791666666666665e-06, 'epoch': 0.013024564328323218}, {'loss': 0.4246, 'grad_norm': 1.0494062900543213, 'learning_rate': 4.1624999999999995e-06, 'epoch': 0.026049128656646436}, {'loss': 0.45

TrainOutput(global_step=38389, training_loss=0.475953300329206, metrics={'train_runtime': 27464.6412, 'train_samples_per_second': 11.182, 'train_steps_per_second': 1.398, 'total_flos': 4.870412890759987e+16, 'train_loss': 0.475953300329206, 'epoch': 1.0})

In [ ]:
print("---Saving model---")
path_to_drive = "model_" + transl_direction
model.save_pretrained(path_to_drive, from_pt=True)
print("---Model saved---")

---Saving model---
---Model saved---


In [ ]:
print(f"---history of {transl_direction} training---")
print(trainer_ru_tat.state.log_history)
print("---Done---")

---history of ru->tat training---
[{'loss': 0.4941, 'grad_norm': 0.8615570664405823, 'learning_rate': 2.0791666666666665e-06, 'epoch': 0.013024564328323218, 'step': 500}, {'loss': 0.4246, 'grad_norm': 1.0494062900543213, 'learning_rate': 4.1624999999999995e-06, 'epoch': 0.026049128656646436, 'step': 1000}, {'loss': 0.4508, 'grad_norm': 0.9204182624816895, 'learning_rate': 6.245833333333333e-06, 'epoch': 0.039073692984969655, 'step': 1500}, {'loss': 0.452, 'grad_norm': 0.6679643988609314, 'learning_rate': 7.999907373652774e-06, 'epoch': 0.05209825731329287, 'step': 2000}, {'loss': 0.4489, 'grad_norm': 0.6889986395835876, 'learning_rate': 7.995025503590974e-06, 'epoch': 0.06512282164161609, 'step': 2500}, {'loss': 0.4561, 'grad_norm': 1.0017911195755005, 'learning_rate': 7.982733185646518e-06, 'epoch': 0.07814738596993931, 'step': 3000}, {'eval_loss': 0.49325963854789734, 'eval_bleu': 26.1806, 'eval_gen_len': 21.4431, 'eval_runtime': 612.3163, 'eval_samples_per_second': 6.827, 'eval_step